Creating Reviewer-Recipe-Rating matrix :

In [61]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [62]:
final_processed_dataset=pd.read_csv(r"Final_Processed_Dataset.csv")

reviewer_recipe_rating_matrix = final_processed_dataset.pivot_table(index='ReviewerName', columns='Name', values='Rating')

Normalising rating of each reviewer's rating to penalise ratings of those who ovverrate/underrate irratically:

In [63]:
def normalise_rating(user_ratings):
    return user_ratings/user_ratings.sum()*user_ratings.count()

In [64]:
reviewer_recipe_rating_matrix=reviewer_recipe_rating_matrix.apply(normalise_rating,axis=1)

Calculating cosine similarity matrix :

In [65]:
reviewer_recipe_rating_relation_matrix=reviewer_recipe_rating_matrix.fillna(0)

from scipy.sparse import csr_matrix

reviewer_recipe_rating_relation_matrix=csr_matrix(reviewer_recipe_rating_relation_matrix.values)
cosine_similarity_matrix=cosine_similarity(reviewer_recipe_rating_relation_matrix)
cosine_similarity_matrix = pd.DataFrame(cosine_similarity_matrix, index=reviewer_recipe_rating_matrix.index, columns=reviewer_recipe_rating_matrix.index)

Predicting Ratings for all recipes where rating is nan for a each user one by one :

In [66]:
# user=input("Tell your ReviewerName : ")
user="Bergy"

user_index=cosine_similarity_matrix.index.get_loc(user)
user_similarity=cosine_similarity_matrix[user]
user_similarity.drop(user,inplace=True)
# user_similarity=user_similarity/user_similarity.sum()  ---Wrong. weights.sum() should not include where nan values
user_similarity=user_similarity.reset_index()
reviewer_recipe_rating_matrix_temp=reviewer_recipe_rating_matrix.reset_index()

Removing already rated/eaten dishes from possible recommendations :

In [67]:
user_data=reviewer_recipe_rating_matrix_temp[reviewer_recipe_rating_matrix_temp["ReviewerName"]==user]
user_data=user_data.drop("ReviewerName",axis=1)
already_rated = user_data.loc[:, user_data.notna().any()]
reviewer_recipe_rating_matrix_temp.drop(columns=already_rated.columns.tolist(),inplace=True)

Applying appropropriate weights to ratings by other users as per similarity :

In [68]:
def apply_similarity_weights(row):
    return row*row.loc[user]

In [69]:
merged_data=pd.merge(user_similarity,reviewer_recipe_rating_matrix_temp,on="ReviewerName")
merged_data.drop("ReviewerName",axis=1,inplace=True)
merged_data=merged_data.apply(apply_similarity_weights,axis=1)

Normalising the weights :

In [70]:
def weighted_ratings(column):
    return column/column.count()

In [71]:
merged_data=merged_data.apply(weighted_ratings,axis=0)
merged_data.drop(user,axis=1,inplace=True)

Final Recommendations :

In [72]:
top_20_recommend=merged_data.sum(axis=0).sort_values(ascending=False).iloc[0:20]
print(top_20_recommend.reset_index()["index"])

0    B
1    C
2    D
Name: index, dtype: object


In [78]:
print('''
Specially_Made_Dataset_to_demonstrate_collaborative_accuracy1 : If we input user=3 (yes numerical) and this file name then->\n
Will predict 1st -> C and then 2nd -> D.\n
Because 3 has similar rating pattern as users 1,2. They gave higher rating to food C rather than D. \n
Exact opposite can be said for users 4,5. Hence D did not prevail over C even though they gave 5 to it. \n
\n
Specially_Made_Dataset_to_demonstrate_collaborative_accuracy2 : If we input user=3 and this file name then ->\n
Will predict 1st -> B and then 2nd -> C and 3rd -> D.\n
Beacuse 3 has similar rating pattern as users 1,2. They gave higher rating to food B rather than C rather than D.\n
Exact opposite can be said for users 4,5. Hence D did not prevail over C, C did not prevail over B even though 4,5 gave higher rating to it.\n
''')


Specially_Made_Dataset_to_demonstrate_collaborative_accuracy1 : If we input user=3 (yes numerical) and this file name then->

Will predict 1st -> C and then 2nd -> D.

Because 3 has similar rating pattern as users 1,2. They gave higher rating to food C rather than D. 

Exact opposite can be said for users 4,5. Hence D did not prevail over C even though they gave 5 to it. 



Specially_Made_Dataset_to_demonstrate_collaborative_accuracy2 : If we input user=3 and this file name then ->

Will predict 1st -> B and then 2nd -> C and 3rd -> D.

Beacuse 3 has similar rating pattern as users 1,2. They gave higher rating to food B rather than C rather than D.

Exact opposite can be said for users 4,5. Hence D did not prevail over C, C did not prevail over B even though 4,5 gave higher rating to it.


